This file is WGAN+DCGAN with gradient penalty.

The input of this WGAN model are noise and historical features extracted by middle layers of VGG16.

The first section mounts google drive, and load historical/future candlestick images from bucket.

The second part imports all the essential libaraies.

The third and forth sections create generator, discriminator, and VGG models.

Block number five is used to save and load checkpoint from google drive.

From section six to eight are the functions used to load dataset and store the training results.

The ninth section includes the functions which are used to training WGAN.

The final block is some parameters used for training.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

!mkdir historical
!mkdir future
!gsutil -m cp gs://ganstick_project/historical/*.png historical/ &> /dev/null
!gsutil -m cp gs://ganstick_project/future/*.png future/ &> /dev/null

In [ ]:
!pip install imageio git+https://github.com/tensorflow/docs XlsxWriter tensorflow_addons &> /dev/null

import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL

import tensorflow.keras as keras
import tensorflow_addons as tfa
from tensorflow_addons.layers import SpectralNormalization as SpectralNorm
from tensorflow.keras import Model

from keras.preprocessing.image import load_img
from keras_preprocessing.image import ImageDataGenerator

from keras import backend as bk
from keras.models import Sequential
from keras.layers import Input, InputSpec, Layer, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, Rescaling
from keras import initializers, regularizers, constraints

import time

from IPython import display
import tensorflow_docs.vis.embed as embed
import xlsxwriter

from sklearn.preprocessing import StandardScaler

In [ ]:
# models
def make_generator_model():
  # create noise and reshape
  input_noise = Input(shape=(100,))
  n_nodes = 256 * 7 * 7
  noise = Dense(n_nodes)(input_noise)

  gen_image = Reshape((7, 7, 256))(noise)

  gen_image = BatchNormalization()(gen_image)
  gen_image = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', use_bias=False)(gen_image)
  gen_image = LeakyReLU()(gen_image)

  gen_image = BatchNormalization()(gen_image)
  gen_image = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', use_bias=False)(gen_image)
  gen_image = LeakyReLU()(gen_image)
  
  gen_image = BatchNormalization()(gen_image)
  gen_image = Conv2DTranspose(3, (3, 3), strides=(2, 2), padding='same', use_bias=False, activation=keras.activations.tanh)(gen_image)
  
  model = Model(inputs=input_noise, outputs=gen_image)
  return model

def make_critic_model():
  input_image = Input(shape=(56, 56, 3))

  image = Conv2D(32, (3, 3), strides=(2, 2), padding='same')(input_image)
  image = LeakyReLU()(image)

  image = Conv2D(64, (3, 3), strides=(2, 2), padding='same')(image)
  image = LeakyReLU()(image)

  feature = Flatten()(image)
  
  # critic needs linear output
  prediction = Dense(1)(feature)

  model = Model(inputs=input_image, outputs=prediction)
  return model

generator = make_generator_model()
critic = make_critic_model()

# loss functions
def generator_loss_fn(fake_img):
  return -tf.reduce_mean(fake_img)

def critic_loss_fn(real_img, fake_img):
  real_loss = tf.reduce_mean(real_img)
  fake_loss = tf.reduce_mean(fake_img)
  return fake_loss - real_loss

# optimizers
generator_optimizer = keras.optimizers.Adam(
  learning_rate=2e-4, beta_1=0.5, beta_2=0.9
)

critic_optimizer = keras.optimizers.Adam(
  learning_rate=2e-4, beta_1=0.5, beta_2=0.9
)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

vgg_layers = [l for l in vgg.layers]
input = keras.layers.Input(shape=(56,56,3))
x = keras.layers.Conv2D(128, (1,1), strides=(1,1), padding='same')(input)
for i in range(7, len(vgg_layers)-10):
  vgg_layers[i].trainable = False
  x = vgg_layers[i](x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(50)(x)

new_vgg = keras.Model(inputs=input, outputs=x)

In [ ]:
local_device_option = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")

checkpoint_dir = 'drive/MyDrive/AttentionGan/WGAN_checkpoint/vgg5050_gp'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(
  generator=generator,
  critic=critic,
  generator_optimizer=generator_optimizer,
  critic_optimizer=critic_optimizer
)

manager = tf.train.CheckpointManager(checkpoint, checkpoint_prefix, max_to_keep=3)
checkpoint.restore(manager.latest_checkpoint, options=local_device_option)

In [ ]:
# decode for ticker string
def recover_string_from_int(x):
  recoveredbytes = x.to_bytes((x.bit_length() + 7) // 8, 'little')
  recoveredstring = recoveredbytes.decode('utf-8')
  return recoveredstring

def save_result(critic_losses, generator_losses, real_images, time_series_data, generated_images, fin_epoch):
  wb = xlsxwriter.Workbook(f'drive/MyDrive/AttentionGan/WGAN_results/epoch{fin_epoch:03}.xlsx')
  os.makedirs(f'drive/MyDrive/AttentionGan/WGAN_results/epoch{fin_epoch:03}/real', exist_ok=True)
  os.makedirs(f'drive/MyDrive/AttentionGan/WGAN_results/epoch{fin_epoch:03}/generated', exist_ok=True)
  os.makedirs(f'drive/MyDrive/AttentionGan/WGAN_results/epoch{fin_epoch:03}/timeseriesdata', exist_ok=True)
  ws = wb.add_worksheet()
  ws.write_row(0, 0, ('Batch Index', 'Critic Loss', 'Generator Loss'))
  batch_num = 1
  for critic_loss, gen_loss in zip(critic_losses, generator_losses):
    ws.write_row(batch_num, 0, (batch_num, critic_loss, gen_loss))
    batch_num += 1

  for real_img, timedata, gen_img in zip(real_images, time_series_data, generated_images):
    timedata = timedata.reshape((1,5))
    # kinda janky, to get ticker integer encoding
    ticker = recover_string_from_int(int(timedata[0][-1]))
    first_date = str(timedata[0][1])
    last_date = str(timedata[0][2])

    df = pd.DataFrame(timedata, columns=['avg_volatility', 'first_date', 'last_date', 'avg_volume', 'ticker'])

    # save ticker name as csv name, then timeseries in csv
    df.to_csv(f'drive/MyDrive/AttentionGan/WGAN_results/epoch{fin_epoch:03}/timeseriesdata/{ticker}_{first_date}_{last_date}.csv', index=False)

    save_img = (real_img * 127.5 + 127.5)
    save_img = PIL.Image.fromarray(np.uint8(save_img))
    save_img.save(f'drive/MyDrive/AttentionGan/WGAN_results/epoch{fin_epoch:03}/real/{ticker}_{first_date}_{last_date}.png') 

    save_img = (gen_img * 127.5 + 127.5)
    save_img = PIL.Image.fromarray(np.uint8(save_img))
    save_img.save(f'drive/MyDrive/AttentionGan/WGAN_results/epoch{fin_epoch:03}/generated/{ticker}_{first_date}_{last_date}.png')
  wb.close()

In [ ]:
tickers = [
  'aapl',
  'mcd',
  'pld',
  'bac',
  'cvx',
  'ibm',
  'v',
  'pg',
  'nke',
  'abbv',
  'mmm',
  'rio',
  'cci',
  'ip',
  'gs',
  'hon',
  'msft',
  'amt',
  'spg',
  'jpm',
  'amzn',
  'unh',
  'wmt',
  'jnj',
  'vz',
  'bhp',
  'nee',
  'etr',
  'xel',
  'pfe',
  'xom',
  'lmt',
  'duk',
  'googl',
  'viac',
  'intc',
  'ko',
  ]
future_vols = []
hist_vols = []
for ticker in tickers:
  fname = glob.glob("drive/MyDrive/ganstick_project/future_volatility/%s/%s_future_vol.csv"%(ticker, ticker))[0]
  df = pd.read_csv(fname)
  df['id'] = range(0, len(df))
  # super super janky but possibly only way to make image data generator figure it out
  df['id'] = df['id'].apply(lambda x: 'future/%s_'%ticker + str(x) + '.png')
  future_vols.append(df)

  # just to be sure
  del df

  fname = glob.glob("drive/MyDrive/ganstick_project/historical_volatility/%s/%s_hist_vol.csv"%(ticker, ticker))[0]
  df = pd.read_csv(fname)
  df['id'] = range(0, len(df))
  # super super janky but possibly only way to make image data generator figure it out
  df['id'] = df['id'].apply(lambda x: 'historical/%s_'%ticker + str(x) + '.png')
  hist_vols.append(df)

all_hist_vols = pd.concat([df for df in hist_vols])
all_fut_vols = pd.concat([df for df in future_vols])

In [ ]:
# as large as possible a batch size as we can fit in memory for GAN
BATCH_SIZE = 256

def process(image):
  image = tf.cast((image-127.5) / 127.5 ,tf.float32)
  return image

image_gen = ImageDataGenerator(
    preprocessing_function=process
)

# class_mode == raw --> pass in multiple columns to y_col to add
hist_gen = image_gen.flow_from_dataframe(
    dataframe=all_hist_vols,
    directory=None,
    x_col='id',
    y_col=['avg_vol', 'first_date', 'last_date', 'avg_volume', 'ticker'],
    target_size=(56,56),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='raw',
)

fut_gen = image_gen.flow_from_dataframe(
    dataframe=all_fut_vols,
    directory=None,
    x_col='id',
    y_col=['avg_vol', 'first_date', 'last_date', 'avg_volume', 'ticker'],
    target_size=(56,56),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='raw',
)

In [ ]:
@tf.function
def gradient_penalty(batchsize, critic, real_images, fake_images):
  alpha = tf.random.normal([batchsize, 1, 1, 1], 0.0, 1.0)
  diff = fake_images - real_images
  interpolated = real_images + alpha * diff
  with tf.GradientTape() as gp_tape:
    gp_tape.watch(interpolated)
    # 1. Get the critic output for this interpolated image.
    pred = critic(interpolated, training=False)
  # 2. Calculate the gradients w.r.t to this interpolated image.
  grads = gp_tape.gradient(pred, [interpolated])[0]
  # 3. Calculate the norm of the gradients.
  norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
  gp = tf.reduce_mean((norm - 1.0) ** 2)
  return gp


@tf.function
def trainstep_critic(generator, critic, hist_imgs, fut_imgs, current_batchsize, latent_dim, gradient_penalty_weight):
  with tf.GradientTape() as tape:
    noise = tf.random.normal([current_batchsize, latent_dim])
    hist_noise = new_vgg(hist_imgs, training=False)
    noise = tf.concat([noise, hist_noise], -1)
    fake_images = generator(noise, training=True)

    fake_logits = critic(fake_images, training=True)
    real_logits = critic(fut_imgs, training=True)

    critic_cost = critic_loss_fn(real_img=real_logits, fake_img=fake_logits)
    gp = gradient_penalty(current_batchsize, critic, fut_imgs, fake_images)
    critic_loss = critic_cost + (gp * gradient_penalty_weight)
      
  critic_gradient = tape.gradient(critic_loss, critic.trainable_variables)
  critic_optimizer.apply_gradients(zip(critic_gradient, critic.trainable_variables))

  return critic_loss


@tf.function
def trainstep_generator(generator, critic, hist_imgs, fut_imgs, current_batchsize, latent_dim, gradient_penalty_weight):
  with tf.GradientTape() as tape:
    noise = tf.random.normal([current_batchsize, latent_dim])
    hist_noise = new_vgg(hist_imgs, training=False)
    noise = tf.concat([noise, hist_noise], -1)
    fake_images = generator(noise, training=True)

    generated_logits = critic(fake_images, training=True)
    generator_loss = generator_loss_fn(generated_logits)
        
  generator_gradient = tape.gradient(generator_loss, generator.trainable_variables)
  generator_optimizer.apply_gradients(zip(generator_gradient, generator.trainable_variables))
  
  return generator_loss, fake_images


def train(generator, critic, hist_dataset, fut_dataset, latent_dim, num_epochs, fin_epoch, batchsize, critic_update_rate, gradient_penalty_weight):
  for epoch in range(fin_epoch+1, num_epochs+1):
    start = time.time()
    current_epoch = epoch
    print('Start training for epoch %s'%current_epoch)

    real_images = []
    time_series_data = []
    generated_images = []

    critic_losses = []
    generator_losses = []

    # dataset has been split into batches already (so len is batch per)
    batches_per_dataset = len(hist_dataset)

    for i in range(batches_per_dataset):
      hist_imgs, time_series = next(hist_dataset)
      fut_imgs, _ = next(fut_dataset)

      current_batchsize = hist_imgs.shape[0]

      critic_loss = trainstep_critic(generator, critic, hist_imgs, fut_imgs, current_batchsize, latent_dim, gradient_penalty_weight)

      # generator training
      if (i+1) % critic_update_rate == 0:
        generator_loss, fake_images = trainstep_generator(generator, critic, hist_imgs, fut_imgs, current_batchsize, latent_dim, gradient_penalty_weight)

        critic_losses.append(critic_loss)
        generator_losses.append(generator_loss)

        for j in range(3):
          real_images.append(fut_imgs[j])
          time_series_data.append(time_series[j])
          generated_images.append(fake_images[j])

      if (i+1) % 15 == 0:
        print("Batch %s finished"%(i+1))
        

    manager.save(options=tf.train.CheckpointOptions(experimental_io_device="/job:localhost"))

    save_result(critic_losses, generator_losses, real_images, time_series_data, generated_images, current_epoch)

    display.clear_output(wait=True)
    print('Time for epoch {} is {} sec'.format(current_epoch, time.time()-start))
    print('latest generator loss:', generator_losses[-1])
    print('latest critic loss:', critic_losses[-1])

In [ ]:
'''
Wasserstein Loss
Primary importance is the trend;
looking for critic_fake loss to trend downward --> higher quality generated images
'''
noise_dim = 50
critic_update_rate = 1
gradient_penalty_weight = 10

target_epochs = 200
finished_epochs = 0

train(generator, critic, hist_gen, fut_gen, noise_dim, target_epochs, finished_epochs, BATCH_SIZE, critic_update_rate, gradient_penalty_weight)